Created 09/01/2026 by truji@mit.edu

A simple candidate filter for picking targets. 
Takes observatory code and list of possible targets from, in my case, the NEA Observation Planning Aid, and narrows them down based on observing window etc.
Uses JPL Horizons to do this.

In [1]:
from astroquery.jplhorizons import Horizons
from astropy import units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord, get_body, get_sun
from astropy.time import Time
import re
from tqdm import tqdm

In [2]:
# make list of object names by copying from NEA Observation Planning Aid output

file_path = "possible_targets.txt"
with open(file_path, "r") as file:
    raw = file.read()

ids = [int(m.group(1)) for m in re.finditer(r'(?m)^\s*\((\d+)\)', raw)]

In [5]:
# set obs time
utc = Time("2026-01-21 22:00:00", scale='utc')

# get target altitudes
all = {}
for n, name in tqdm(enumerate(ids), total=len(ids)):
    obj = Horizons(name, location=954, epochs=[utc.tdb.jd])
    tab = obj.ephemerides()

    # Horizons package can't handle physical properties. Find rotper with regex on raw api out
    raw = str(obj.ephemerides_async()._content)
    match = re.search(r"ROTPER=\s*([0-9.]+)", raw)
    rotper = float(match.group(1)) if match else 1e6

    # update dict with all needed info
    all[name] = (float(tab['EL'][0]), float(tab['hour_angle'][0]), float(tab['V'][0]), rotper)

100%|██████████| 310/310 [07:26<00:00,  1.44s/it]


In [6]:
# review top candidates. best is determined by alt / mag
cutoff = sorted([value[0] / value[2] for value in all.values()])[-20]

best_targets = {} # save as csv using vscode datawrangler
for name, value in all.items():
    if value[0] / value[2] >= cutoff and value[3] == 1e6:
        print(f"{name} will be at alt {value[0]} and hour angle {value[1]} at {utc}. It is magnitude {value[2]} and has rotper {value[3]}")
        best_targets[name] = value

367390 will be at alt 79.352349 and hour angle -0.687688952 at 2026-01-21 22:00:00.000. It is magnitude 17.625 and has rotper 1000000.0
207970 will be at alt 72.33848 and hour angle 0.329469104 at 2026-01-21 22:00:00.000. It is magnitude 17.936 and has rotper 1000000.0
847304 will be at alt 82.155279 and hour angle -0.172895544 at 2026-01-21 22:00:00.000. It is magnitude 18.789 and has rotper 1000000.0
331509 will be at alt 76.145982 and hour angle 0.262421005 at 2026-01-21 22:00:00.000. It is magnitude 19.497 and has rotper 1000000.0
190161 will be at alt 80.978953 and hour angle 0.630921629 at 2026-01-21 22:00:00.000. It is magnitude 19.368 and has rotper 1000000.0
348776 will be at alt 84.029525 and hour angle 0.428827235 at 2026-01-21 22:00:00.000. It is magnitude 20.014 and has rotper 1000000.0
816502 will be at alt 75.394396 and hour angle -1.105908299 at 2026-01-21 22:00:00.000. It is magnitude 20.322 and has rotper 1000000.0
467609 will be at alt 80.025164 and hour angle 0.5352